# BertSum

# Make datasets

## Import library

In [1]:
%load_ext autoreload
%autoreload

In [1]:
import os
import sys
import json
from glob import glob
import pickle
from typing import Collection

import pandas as pd
import numpy as np
from tqdm import tqdm

tqdm.pandas()

PROJECT_DIR = os.path.abspath("..")
print(PROJECT_DIR)
sys.path.append(PROJECT_DIR + "/src")

/home/link/Project/KoBertSum


In [2]:
# 아래는 한 번만 실행하기
os.chdir(os.path.abspath(PROJECT_DIR + "/src"))
print(os.getcwd())

/home/link/Project/KoBertSum/src


## Preprocessing

In [4]:
# !pip install -r requirements_prepro.txt

### Import data as dataframe and export as pkl

In [6]:
from prepro.utils import load_obj, save_obj

In [7]:
# Load jsonl file
train_data = load_obj(PROJECT_DIR + "/datasets/sample/sample_train.jsonl")
test_data = load_obj(PROJECT_DIR + "/datasets/sample/sample_test.jsonl")
train_data[0]

Loaded from /home/link/Project/KoBertSum/datasets/sample/sample_train.jsonl
Loaded from /home/link/Project/KoBertSum/datasets/sample/sample_test.jsonl


{'text': ['당진시 문화관광과를 대상으로 하는 행정사무감사에서 당진시립합창단 관계자가 보낸 것으로 추정되는 문자와 관련해 당진시의회가 행정사무조사특별위원회를 구성해 조사하겠다고 밝혔다.',
  '당진시의회 행정사무감사 3일차였던 지난 6일 문화관광과를 대상으로 하는 행감에서 최창용 의원은 “(당진시립합창단 정기연주회를 앞두고) 문자메세지를 제보받았다”며 “(음향팀에 보낸 것으로 추정되는) 해당 문자에는 ‘합창단이 소리를 작게 낼 것이니 알고 있으라’는 내용이었다”고 말했다.',
  '이어 “공연에서 소리를 작게 낸다는 것은 합창단으로서 그 임무를 하지 않겠다는 것”이며 “공연 자체를 무력화해 당진시를 망신 주려는 행위”라며 해당 문자를 보낸 단원 등 연루된 사람들을 찾아 사실관계를 확인하고 징계 등 책임을 물어야 한다고 지적했다.',
  '문제의 문자메세지를 전달받은 문화관광과는 감사법무담당관에게 조사를 의뢰했다.',
  '권경선 문화관광과장은 “누가, 어떻게 해서 해당 문자가 나온 것인지 정확히 조사해봐야 알 수 있다”며 “전달받은 문자 내용도 최 의원이 언급한 부분만 있어, 중간 내용만을 가지고는 전체를 유추할 수 없다”고 전했다.',
  '하지만 감사법무담당관실에서 아직 조사가 이뤄지지 않고 있어 당진시가 사태의 심각성을 인지하지 못하고 있다는 지적이다.',
  '그동안 행정사무감사가 진행되고 있어 사태를 지켜봤다던 감사법무담당관실에서는 “관계된 사람들을 조사해 사태를 파악해야 하는데, 아직 조사에 대한 뚜렷한 계획이 없다”고 답했다.',
  '한편 행감이 끝난 지난 12일 당진시의회에서는 당진시립합창단 문제를 비롯해 구체적인 조사가 필요한 부분에 대해 행정사무조사특별위원회를 구성해 운영하겠다고 밝혔다.',
  '김기재 의장은 “본회의 의결과 제적의원 1/3의 발의가 있으면 행정사무조사특별위원회를 구성할 수 있다”며 “다음 달 초 위원들과 상의해 위원회를 구성한다면 당진시립합창단 관련 사안을 비롯해 사회복지기관 위수탁 등에 대해 다룰 계획”이라고 

In [8]:
# Convert jsonl to df
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)
train_df.head()

,text,summary
0,[당진시 문화관광과를 대상으로 하는 행정사무감사에서 당진시립합창단 관계자가 보낸 것...,지난 6일 당진시의회 행정사무감사에서 '합창단이 소리를 작게 낼 것이니 알고 있으라...
1,[미국 메이저리그(MLB)에서 활동하는 한국 선수들의 시즌 초반 희비가 엇갈리고 있...,LA 에인절스의 최지만이 맹활약을 하여 시즌 타율 0.250에서 0.313으로 올리...
2,[당진시 문화관광과를 대상으로 하는 행정사무감사에서 당진시립합창단 관계자가 보낸 것...,지난 6일 당진시의회 행정사무감사에서 '합창단이 소리를 작게 낼 것이니 알고 있으라...
3,[미국 메이저리그(MLB)에서 활동하는 한국 선수들의 시즌 초반 희비가 엇갈리고 있...,LA 에인절스의 최지만이 맹활약을 하여 시즌 타율 0.250에서 0.313으로 올리...
4,[당진시 문화관광과를 대상으로 하는 행정사무감사에서 당진시립합창단 관계자가 보낸 것...,지난 6일 당진시의회 행정사무감사에서 '합창단이 소리를 작게 낼 것이니 알고 있으라...


In [9]:
 # random split
df = train_df.copy()
train_df = df.sample(frac=0.9, random_state=42)  # random state is a seed value
valid_df = df.drop(train_df.index)
train_df.reset_index(inplace=True, drop=True)
valid_df.reset_index(inplace=True, drop=True)
valid_df

In [11]:
train_df.to_pickle(PROJECT_DIR + "/datasets/sample/df/train_df.pkl")
valid_df.to_pickle(PROJECT_DIR + "/datasets/sample/df/valid_df.pkl")
test_df.to_pickle(PROJECT_DIR + "/datasets/sample/df/test_df.pkl")

### df to Bert data

In [30]:
# in src folder... run below code
!python make_data.py \
  dataset=sample \
  dataset.text.name=text dataset.text.type=str_list dataset.text.do_cleaning=True \
  dataset.summary.name=summary dataset.summary.type=str \
  dataset.is_informal=False \
  model_name="kykim/bert-kor-base" n_cpus=28
  
# Above code is same with below, because of 
# conf/make_data/config.yaml and conf/make_data/dataset/sample.yaml
# !python make_data.py dataset=sample

/home/link/miniconda3/envs/kobertsum/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/home/link/Project/KoBertSum/datasets/sample
['df/test_df.pkl', 'df/valid_df.pkl', 'df/train_df.pkl']
[2021-10-04 07:46:38,027 INFO] json folder is removed
[2021-10-04 07:46:38,027 INFO] json folder is made
[2021-10-04 07:46:38,031 INFO] bert folder is removed
[2021-10-04 07:46:38,031 INFO] bert folder is made
[2021-10-04 07:46:38,031 INFO] Start 'df_to_bert' processing for df/test_df.pkl
[2021-10-04 07:46:38,032 INFO] Cleaning...
100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 387.55it/s]
[2021-10-04 07:46:38,038 INFO] Sent split...
100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 1158.33it/s]
[2021-10-04 07:46:38,040 INFO] bert/test folder is made
[('test', '